In [0]:
# IMPORT LIBRARIES

from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import Adam, SGD
from keras.layers.advanced_activations import LeakyReLU

import pandas as pd
import numpy as np

In [0]:
# READ DATA
features_df = pd.read_csv('features.csv')
labels_df = pd.read_csv('labels.csv')
labels = labels_df.ix[327:653,:].values.astype('int32')
X_train = (features_df.ix[327:653,:].values).astype('float32')
# X_test = (pd.read_csv('test.csv').values).astype('float32')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [0]:
# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(labels) 

# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
# X_test /= scale

mean = np.std(X_train)
X_train -= mean
# X_test -= mean

input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

In [0]:
# Here's a Deep Dumb MLP (DDMLP)
l = LeakyReLU(alpha=0.3)
model = Sequential()
model.add(Dense(70, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.65))
model.add(Dense(60))
model.add(Activation('relu'))
model.add(Dropout(0.65))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.65))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_213 (Dense)            (None, 70)                15330     
_________________________________________________________________
activation_213 (Activation)  (None, 70)                0         
_________________________________________________________________
dropout_159 (Dropout)        (None, 70)                0         
_________________________________________________________________
dense_214 (Dense)            (None, 60)                4260      
_________________________________________________________________
activation_214 (Activation)  (None, 60)                0         
_________________________________________________________________
dropout_160 (Dropout)        (None, 60)                0         
_________________________________________________________________
dense_215 (Dense)            (None, 50)                3050      
__________

In [0]:
# we'll use categorical xent for the loss, and RMSprop as the optimizer
# opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.00)
sgd = SGD(lr=0.1, momentum=0.9, nesterov=True)
# lss = keras.losses.kullback_leibler_divergence(y_true, y_pred)
model.compile(loss= 'categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

print("Training...")
model.fit(X_train, y_train, nb_epoch=5, batch_size=16, validation_split=0, verbose=2)

Training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/5
 - 14s - loss: 0.6996 - acc: 0.5015
Epoch 2/5
 - 0s - loss: 0.6994 - acc: 0.4924
Epoch 3/5
 - 0s - loss: 0.6841 - acc: 0.5566
Epoch 4/5
 - 0s - loss: 0.7007 - acc: 0.5138
Epoch 5/5
 - 0s - loss: 0.7023 - acc: 0.4801


In [0]:
test_features_df = pd.read_csv('test_features.csv')
test_labels_df = pd.read_csv('test_labels.csv')
y_test = test_labels_df.ix[231:474,:].values.astype('int32')
X_test = (test_features_df.ix[231:474,:].values).astype('float32')
X_test /= scale
X_test -= mean
# y_test = np_utils.to_categorical(tl) 

print("Generating test predictions...")
preds = model.predict_classes(X_test, verbose=0)
print(preds)
print(y_test.T[0])
a = (y_test.T[0] == preds)
print(a)
print(np.mean(a))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


Generating test predictions...
[1 1 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 0 1 1 0 1
 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0 1
 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 1
 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 0 1 0 1 1 0 0 0 1 0
 1 1 1 0 0 0 1 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1
 1 0 0 1 0 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1]
[0 1 0 0 0 1 0 1 1 0 1 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1
 0 0 1 0 0 0 0 0 1 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 0 0 1 1 1 0 1 0 0 1 1 0 1
 0 0 0 0 1 1 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0
 1 1 0 0 0 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 1 0 0 1 1 0 0 1 1 0 1 0 1
 0 0 0 0 0 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 1 1 0
 1 1 1 0 1 0 0 1 1 0 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 1 0
 0 1 0 0 1 1 0 1 0 0 0 